# Example usage of `sqlcompyre`

This notebook will demonstrate and explain some example usage of the `sqlcompyre` tool.

In [ ]:
import sqlalchemy as sa
import pandas as pd
import copy
import sqlcompyre as sc

## Creating Tables

Here we set up a connection using sqlite and create a few test tables to compare. In most use cases, the tables being compared will already exist, so the next cell can be ignored

In [ ]:
eng = sa.create_engine('sqlite:///example.sqlite3')
base_students = [
    {"id": 1, "name": "Connor", "age": 22, "gpa": 2.57},
    {"id": 2, "name": "Sophie", "age": 20, "gpa": 3.45},
    {"id": 3, "name": "Sam", "age": 17, "gpa": 3.55},
    {"id": 4, "name": "Chris", "age": 51, "gpa": 3.88},
    {"id": 5, "name": "Sydney", "age": 52, "gpa": 3.95},
    {"id": 6, "name": "Kevin", "age": 19, "gpa": 3.85},
    {"id": 7, "name": "Chris", "age": 22, "gpa": 3.62},
    {"id": 8, "name": "Alyssa", "age": 23, "gpa": 3.97},
    {"id": 9, "name": "Ostap", "age": 21, "gpa": 3.04},
    {"id": 10, "name": "Rokas", "age": 20, "gpa": 3.25},
]

students_1 = base_students[:-1]

students_2 = copy.deepcopy(base_students)[2:]
students_2[1]["name"] = "Christopher"
students_2[4]["name"] = "Christopher"
students_2[2]["name"] = "Syd"
students_2[4]["age"] = 52

column_mapping = {
    "id": "identification",
    "name": "label",
    "age": "wisdom",
    "gpa":  "score"
}
students_2 = [
    {column_mapping[key]: value for key, value in student.items()}
    for student in students_2
]

df1 = pd.DataFrame(students_1)
df1.to_sql("students1", eng, if_exists="replace", index=False)

df2 = pd.DataFrame(students_2)
df2.to_sql("students2", eng, if_exists="replace", index=False)

## Connecting to a Database and Finding Tables

Next, we use SQLAlchemy to connect to our database and find our tables.

In [ ]:
engine = sa.create_engine('sqlite:///example.sqlite3')
meta = sa.MetaData()
meta.reflect(bind=engine)

table1 = meta.tables["students1"]
table2 = meta.tables["students2"]

In [ ]:
df1 = pd.read_sql(sa.select(table1), engine)
df1

In [ ]:
df2 = pd.read_sql(sa.select(table2), engine)
df2

## Creating a TableComparison Object

To begin compare tables, we simply call `compare_tables` which uses the engine we created before in order to access any database objects. In the simplest case, this method call merely takes the names of the two tables to compare. However, in our case, we also have to specify a column to join the tables on (`join_columns`) and a mapping between column names (`column_name_mapping`).

The `compare_tables` method returns a `TableComparison` object which we will work with in the following.

In [ ]:
compare_1_2 = sc.compare_tables(
    engine, table1, table2, join_columns=["id"], column_name_mapping={
        "id": "identification",
        "name": "label",
        "age": "wisdom",
        "gpa": "score"
    }
)
print(compare_1_2)

## Generating Reports

Often the first thing we will want to do with a `TableComparison` is to generate a report to get an idea of what the key differences between the compared tables are. A simple report could look like the following:

In [ ]:
compare_1_2.summary_report()

## Exploring Returned Data

After looking at the report, we may wish to explore some of the values in more detail, or use them programmatically. Some values, such as row counts and column names, are already accessible:

In [ ]:
n_columns_left = compare_1_2.row_counts.left
print(f"{n_columns_left} columns in the left table")

n_rows_unjoined_left = compare_1_2.row_matches.n_unjoined_left
print(f"{n_rows_unjoined_left} columns in the left table could not be inner-joined")

Additionally, some SQLAlchemy queries are stored in these objects that we can use to explore the data in more depth. Since these queries have not yet been executed, we can add on to them!

In [ ]:
unjoined_left = compare_1_2.row_matches.unjoined_left
print(unjoined_left)

# We can use our engine to execute these queries!
unjoined_left_df = pd.read_sql(unjoined_left, engine)
print(unjoined_left_df)

As an example of adding to these queries, let's limit the number of rows we query for to 1

In [ ]:
unjoined_left_limited = unjoined_left.limit(1)

print("Unjoined Left Query (Limited):")
print(unjoined_left_limited)

# We can use our engine to execute these queries!
unjoined_left_df = pd.read_sql(unjoined_left_limited, engine)
print("\nUnjoined Left Query Result (Limited):")
print(unjoined_left_df)

Finally, we can use the function `get_top_changes` to access the most common changes in each column:

In [ ]:
compare_1_2.get_top_changes("name")

Now you should be ready to get started on your own!